In [1]:
!pip install bs4
!pip install -U selenium
!pip install pandas
!pip install webdriver-manager
!pip install service
!pip install setproctitle
!pip3 install webdriver_manager

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1271 sha256=38ff04ce520874c7a1db8d15715d01385b7e8d64234e3d394f7ffc09e28d8a61
  Stored in directory: c:\users\soyeon jang\appdata\local\pip\cache\wheels\73\2b\cb\099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4
  Created wheel for service: filename=service-0.6.0-py3-none-any.whl size=7405 sha256=f2575de24a2eb56637f524fbbd1c1e23d10f28c138c545e06e03addb82d3ce81
  Stored in directory: c:\users\soyeon jang\appdata\local\pip\cache\wheels\51\93\bb\c2361261a209c909774df7d061ec4b577902d10282365e08b4
Successfully built service


In [2]:
from selenium import webdriver
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import os
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

In [111]:
# need to change so that it takes user input (keyword extraction from user resume)
def search_link(job, city, prov):
    url=f'https://www.linkedin.com/jobs/{job}-jobs-{city}-{prov}?trk=homepage-jobseeker_suggested-search&position=1&pageNum=0'
    return url

wd = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
wd.get(search_link('data-scientist', 'waterloo', 'ON'))
# instead of doing a link. find element to click and insert search items
num_jobs = wd.find_element(by=By.CSS_SELECTOR, value='h1>span').get_attribute('innerText')
if num_jobs[-1]=='+':
    num_jobs=int(num_jobs.replace(',', '')[:-1])
else:
    num_jobs=int(num_jobs)
print(num_jobs)

i = 2
list_jobs=[]
while i <= (num_jobs/25)+1: 
    job_lists=wd.find_element(by=By.CLASS_NAME, value='jobs-search__results-list')
    jobs=job_lists.find_elements(by=By.TAG_NAME, value='li') # returns a list
    wd.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    i = i + 1
    try:
        xpath='/html/body/main/div/section/button'
        wd.find_element(by=By.XPATH, value=xpath).click()
    except:
        pass
    
len(jobs)




====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Soyeon Jang\.wdm\drivers\chromedriver\win32\98.0.4758.102]


253


25

In [112]:
job_id= []
job_title = []
company_name = []
location = []
date = []
job_link = []

for job in jobs:
    job_title0 = job.find_element(by=By.CSS_SELECTOR, value='h3').get_attribute('innerText')
    job_title.append(job_title0)
    
    company_name0 = job.find_element(by=By.CSS_SELECTOR, value='h4').get_attribute('innerText')
    company_name.append(company_name0)

    location0 = job.find_element(by=By.CSS_SELECTOR, value='[class=\'job-search-card__location\']').get_attribute('innerText')
    location.append(location0)

    date0=job.find_element(by=By.CSS_SELECTOR, value='div>div>time').get_attribute('datetime')
    date.append(date0)
    
    job_link0=job.find_element(by=By.CSS_SELECTOR, value='a').get_attribute('href')
    job_link.append(job_link0)


In [113]:
jd = []
seniority = []
emp_type = []
job_func = []
industries = []

for item in range(len(jobs)):
    job_func0=[]
    industries0=[]
    # clicking job to view job details
    job_click_path = f'/html/body/div[1]/div/main/section[2]/ul/li[{item+1}]/div/a'
    job_click=job.find_element(by=By.XPATH, value=job_click_path).click()

    jd_path = '/html/body/div[1]/div/section/div[2]/div[1]/section[1]/div/div/section'
    jd0=job.find_element(by=By.XPATH, value=jd_path).get_attribute('innerText')
    jd.append(jd0)
    
    time.sleep(10)

In [117]:
job_data = pd.DataFrame({
    'Date': date,
    'Company': company_name,
    'Title': job_title,
    'Location': location,
    'Description': jd,
    'Link': job_link
})# cleaning description column

job_data['Description'] = job_data['Description'].str.replace('\n',' ')
# job_data.to_excel('LinkedIn Job.xlsx', index = False)

In [116]:
job_data.head()

,Date,Company,Title,Location,Description,Link
0,2022-02-25,Limbik,Data Scientist,"Waterloo, Ontario, Canada",Limbik is the leading information defense syst...,https://ca.linkedin.com/jobs/view/data-scienti...
1,2022-02-27,Praemo,Data Scientist,"Kitchener, Ontario, Canada",Limbik is the leading informat...,https://ca.linkedin.com/jobs/view/data-scienti...
2,2022-03-01,TripleLift,Senior Data Scientist,"Kitchener, Ontario, Canada",Responsibilities Build machine...,https://ca.linkedin.com/jobs/view/senior-data-...
3,2022-01-18,Siemens,Data Scientist,"Kitchener, Ontario, Canada",About TripleLiftTripleLift is ...,https://ca.linkedin.com/jobs/view/data-scienti...
4,2022-02-26,"PerkinElmer, Inc.",Machine Learning Engineer,"Kitchener, Ontario, Canada",Change the future with us.We a...,https://ca.linkedin.com/jobs/view/machine-lear...
